## Tuning of SiPM cut threshold

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import awkward as ak
import os
import pint
import glob
import re
from functools import partial

# I use the SiPM QC Analysis kernel (from .venv/bin)

u = pint.get_application_registry()

from lgdo import lh5
from lgdo.lh5 import LH5Store, LH5Iterator, read, read_as, ls, show
from dspeed.vis.waveform_browser import WaveformBrowser
from legendmeta import LegendMetadata

from latools.utils import get_key_for_rawid, get_detector_system_for_channelname, get_filtered_keys_in_detectorsystem
from latools import core
from latools.browse import BrowseTask, BrowseAnydetTask
from latools.histogram import HistogramTask, Histogram2DTask, CategoricalHistogramTask, CategoricalHistogram2DTask, MultiarrayCategoricalHistogramTask
from latools.counter import CountTask

plt.rcParams["figure.figsize"] = (14, 4)
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = 14

proj_dir = "/mnt/atlas02/projects/legend/sipm_qc"
lmeta = LegendMetadata(os.path.join(proj_dir, "metadata/legend-metadata-schwarz"))
chmap = lmeta.channelmap("20250624T140504Z")

In [ ]:
# custom_chains/dsp-002.yaml
dsp_dir = os.path.join(proj_dir, "manual_dsp/generated/p14r006dsp-dsp002")
dsp_files = glob.glob(dsp_dir+"/l200-*-tier_dsp.lh5")
dsp_files.sort()

raw_dir = os.path.join(proj_dir, "data/tier/raw/lac/p14/r006")

def gimme_raw_filename_from_dsp(dspfilename: str):
    return dspfilename.replace(dsp_dir, raw_dir).replace("tier_dsp", "tier_raw")

In [ ]:
main_loop = partial(core.main_loop, tier_filename_dict=
                    [{"raw": gimme_raw_filename_from_dsp(dsp), "dsp": dsp} for dsp in dsp_files])

In [ ]:
# These 49 rawids I see in the event tier of p14 r006 lac. Some channels existing in raw & dsp (e.g. S054) don't show up here...
rawids_used_in_evt = [1052803, 1052804, 1054401, 1054402, 1054403, 1054404, 1054405, 1056000, 1056001, 1056003, 1056004, 1056005, 1057601, 1057602, 1057603, 1057604, 1057605, 1059200, 1059201, 1059202, 1059204, 1059205, 1060801, 1060803, 1060804, 1060805, 1062400, 1062401, 1062402, 1062403, 1062404, 1062405, 1064000, 1064001, 1064002, 1064003, 1064004, 1065600, 1065601, 1065602, 1065603, 1065604, 1065605, 1067200, 1067201, 1067202, 1067203, 1067204, 1067205]
usable_sipm_keys = get_filtered_keys_in_detectorsystem(chmap, "spms", rawids_used_in_evt)

In [ ]:
# set up chain's input data
inputArraysDef = []

def generate_SiPM_hit_defs():
    ret = []
    for sipm in usable_sipm_keys: # get_keys_in_detectorsystem(chmap, "spms"):
        ret += [(f"{sipm}_wf_lower_hwhm", f"{sipm}/dsp/wf_lower_hwhm"), (f"{sipm}_wf_mode", f"{sipm}/dsp/wf_mode"), (f"{sipm}_wf_min_small", f"{sipm}/dsp/wf_min_small")]
    return ret
inputArraysDef += generate_SiPM_hit_defs()

In [ ]:
# set up processing functions
genArrayDef = []

def generate_SiPM_hit_gen_defs():
    ret = []
    for sipm in usable_sipm_keys: # get_keys_in_detectorsystem(chmap, "spms"):
        ret += [
            ([f"{sipm}_wf_min_small", f"{sipm}_wf_mode"], f"{sipm}_min_rel", lambda x: x[0] - x[1]),
            ([f"{sipm}_min_rel", f"{sipm}_wf_lower_hwhm"], f"{sipm}_any_noise_class", lambda x: (-x[0])/(4*x[1])),
        ]
    return ret
genArrayDef += generate_SiPM_hit_gen_defs()

In [ ]:
outDef = []

In [ ]:
xtalk_counts = MultiarrayCategoricalHistogramTask([f"{s}_any_noise_class" for s in usable_sipm_keys],
                    lambda x: x[0] > 1, keymap_fcn=lambda long: long[:4], logy=True)
outDef.append((xtalk_counts.get_shortnames(), xtalk_counts))

In [ ]:
def investigate_sipm(outDef, spm: str, ) -> None:
    outDef += [
    ([f"{spm}_min_rel"], HistogramTask(-50,50, nbins=200, min_entries_required=20000000, logy=True)),
    ([f"{spm}_wf_lower_hwhm"], HistogramTask(0,10, nbins=200, min_entries_required=20000000, logy=True)),
    ([f"{spm}_min_rel", f"{spm}_wf_lower_hwhm"], Histogram2DTask(-50,50,200, 0,10,200, min_entries_required=20000000, logz=True)),
    ([f"{spm}_min_rel", f"{spm}_wf_lower_hwhm"], HistogramTask(-5,10, nbins=200, fcn=lambda x: x[0]/(4*x[1]), min_entries_required=20000000, logy=True, label=spm)),
    ([f"{spm}_min_rel", f"{spm}_wf_lower_hwhm"], BrowseTask(lambda x: x[0] < (-4*x[1]), spm, max_entries=10, verbosity=-1, title=True)),
    ]

for spm in usable_sipm_keys:
    investigate_sipm(outDef, spm)

In [ ]:
main_loop(inputArraysDef, genArrayDef, outDef, crop=False)